In [ ]:
import serial
import cv2
import numpy as np

# Inisialisasi komunikasi serial
arduino = serial.Serial('COM3', 115200, timeout=1)  # Ganti 'COM3' dengan port Arduino

# Fungsi kirim perintah ke Arduino
def send_command(action, speed=200):
    command = f"{action},{speed}\n"
    arduino.write(command.encode())

# AI Sederhana: Deteksi warna merah untuk kontrol
cap = cv2.VideoCapture(0)  # Kamera default

while True:
    ret, frame = cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Deteksi warna merah
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Hitung posisi objek merah
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        center_x = x + w // 2
        
        # Logika AI: Jika objek di kiri, belok kiri; di kanan, belok kanan
        if center_x < 240:  # 240 = setengah lebar frame (640x480)
            send_command("TURN_LEFT", 150)
        elif center_x > 400:
            send_command("TURN_RIGHT", 150)
        else:
            send_command("MOVE_FORWARD", 200)
    else:
        send_command("STOP")

    # Tampilkan hasil
    cv2.imshow('AI Controller', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
arduino.close()